In [ ]:
from eComply.eComply import eComply
from ParksGIS.ParksGIS import GISFactory, Server, LayerServerGen, LayerQuery, LayerEdits
from pandas import DataFrame
# from Transformer import Transformer

Filters

In [ ]:
from datetime import datetime
from typing import Any, Callable

from pandas import merge


def to_json(obj: Any) -> str:
    return obj.to_json(orient="records", date_format="iso")


def applyEdits(dict: dict) -> dict:
    if dict["edits"] is not None and len(dict["aedits"]) > 0:
        try:
            result = dict["server"].apply_edits(list(dict["edits"].values()))
            print(f"Edits Applied: {len(result)}")
        except Exception as e:
            print(e)
    return dict


def guard_edits(dict: dict, key: str, func: Callable[[dict], Any]) -> dict:
    edits = dict["edits"]
    if 0 == len(edits):
        return dict

    ids = edits[~edits[key].isna()][key]
    if 0 == len(ids):
        return dict

    return func(dict)


def extractChanges(
    repo: Any,
    serverGen: int,
    layerId: int,
    strategy: Callable[[Any, Any], DataFrame],
) -> dict:
    changes = repo.extractChanges([LayerServerGen(layerId, serverGen)])

    dict = {
        "serverGen": changes["layerServerGens"][0]["serverGen"] // 1000,
    }

    objectIds = (
        changes["edits"][0]["objectIds"]["adds"]
        + changes["edits"][0]["objectIds"]["updates"]
    )
    if 0 == len(objectIds):
        dict["edits"] = DataFrame()
    else:
        dict["edits"] = strategy(repo, objectIds)

    return dict


def getChanges(
    serverGen: int,
    strategy: Callable[[datetime], list],
) -> dict[str, Any]:
    fromDateTime = datetime.fromtimestamp(serverGen)
    try:
        return {
            "edits": DataFrame(strategy(fromDateTime))},
        }
    except Exception as e:
        print(e)
        return {}


#######################################################################################


# Sending
def contractExtractChanges(
    repo: Server,
    serverGen: int,
    service: eComply,
) -> dict:
    layerId = 1

    def strategy(repo, objectIds):
        changes = repo.query(
            [
                LayerQuery(
                    layerId,
                    ["*"],
                    "OBJECTID IN ("
                    + ",".join(str(i) for i in objectIds)
                    + ") AND EcomplyContract = 1",
                )
            ]
        )[layerId]

        print(f"Contracts Extracted: {len(changes)}")
        # print(to_json(changes[:1]))
        return changes

    dict = extractChanges(repo, serverGen, layerId, strategy)
    dict["server"] = repo
    dict["service"] = service
    return dict


def contractSendEdits(dict: dict) -> dict:
    print(to_json(dict["edits"][:1]))
    guard_edits(
        dict,
        "OBJECTID",
        lambda dict: dict["service"].postContracts(to_json(dict["edits"][:1])),
    )

    print(f"Contracts Sent: {dict['edits']}")
    return dict


def workOrderExtractChanges(
    repo: Server,
    serverGen: int,
    service: eComply,
) -> dict:
    layerId = 0

    def strategy(repo, objectIds):
        changes = repo.query(
            [
                LayerQuery(
                    layerId,
                    [
                        "InspectionGlobalID",
                        "Type",
                        "Status",
                        "LocationDetails",
                        "ActualFinishDate",
                        "Comments",
                        "Contract",
                        "CancelReason",
                        "GlobalID",
                        "ClosedDate",
                        "ClosedByERN",
                        "ClosedByName",
                        "CancelDate",
                        "CancelByERN",
                        "CancelByName",
                        "CreatedDate",
                        "CreatedBYERN",
                        "CreatedByName",
                        "UpdatedDate",
                        "UpdatedByERN",
                        "UpdatedByName",
                        "WOEntity",
                        "PROJSTARTDATE",
                        "Project",
                        "RecommendedSpecies",
                        "ClosedBySystem",
                        "OBJECTID",
                    ],
                    "OBJECTID IN (" + ",".join(str(i) for i in objectIds) + ")",
                )
            ]
        )[layerId].rename(
            columns={
                "LocationDetails": "Location",
                "GlobalID": "WorkOrderGlobalId",
                "PROJSTARTDATE": "ProjStartDate",
                "RecommendedSpecies": "RecSpecies",
                "OBJECTID": "ObjectId",
            }
        )

        print(f"Work Orders Extracted: {len(changes)}")
        # print(to_json(changes[:1]))
        return changes

    dict = extractChanges(repo, serverGen, layerId, strategy)
    dict["server"] = repo
    dict["service"] = service
    return dict


def hydrateInspection(dict: dict) -> dict:
    key = "InspectionGlobalID"

    def strategy(dict):
        layerId = 4

        edits = dict["edits"]
        inspections = (
            dict["server"]
            .query(
                [
                    LayerQuery(
                        layerId,
                        ["PlantingSpaceGlobalID", "GlobalID"],
                        "GlobalID IN ('"
                        + "','".join(str(id) for id in edits[~edits[key].isna()][key])
                        + "')",
                    )
                ]
            )[layerId]
            .rename(columns={"GlobalID": key})
        )

        edits = dict["edits"] = merge(
            edits,
            inspections,
            on=key,
            how="left",
        )
        del edits[key]

        print(f"Inspections hydrated {len(inspections)}")
        return dict

    return guard_edits(dict, key, strategy)


def hydratePlantingSpace(dict: dict) -> dict:
    key = "PlantingSpaceGlobalID"

    def strategy(dict):
        layerId = 2

        edits = dict["edits"]
        plantingSpaces = (
            dict["server"]
            .query(
                [
                    LayerQuery(
                        layerId,
                        [
                            "ParkName",
                            "ParkZone",
                            "Borough",
                            "CommunityBoard",
                            "BuildingNumber",
                            "StreetName",
                            "CityCouncil",
                            "StateAssembly",
                            "GISPROPNUM",
                            "CrossStreet1",
                            "CrossStreet2",
                            "PlantingSpaceOnStreet",
                            "ObjectID",
                            "GlobalID",
                        ],
                        "GlobalID IN ('"
                        + "','".join(str(id) for id in edits[~edits[key].isna()][key])
                        + "')",
                    )
                ]
            )[layerId]
            .rename(
                columns={
                    "GlobalID": "PlantingSpaceGlobalID",
                    "OBJECTID": "PlantingSpaceId",
                    "PlantingSpaceOnStreet": "OnStreetSite",
                }
            )
        )

        dict["edits"] = merge(
            edits, plantingSpaces, on="PlantingSpaceGlobalID", how="left"
        )

        print(f"Planting Spaces hydrated {len(plantingSpaces)}")
        return dict

    return guard_edits(dict, key, strategy)


def workOrderSendEdits(dict: dict) -> dict:
    # print(to_json(dict["edits"][:1]))
    guard_edits(
        dict,
        "PlantingSpaceGlobalID",
        lambda dict: dict["service"].postWorkOrders(to_json(dict["edits"][:1])),
    )

    print(f"Work Orders Sent: {len(dict['edits'])}")
    return dict


# Receiving
def workOrderGetChanges(
    service: eComply,
    repo: Server,
    serverGen: int,
) -> dict:
    layerId = 0

    def strategy(fromDateTime: datetime):
        edits = service.getWorkOrders(fromDateTime)
        print(f"WorkOrders Recieved: {len(edits)}")
        return edits

    dict = getChanges(serverGen, strategy)
    dict["server"] = repo

    return dict


def update_associated_inspection_HasActiveWorkOrder(dict: dict) -> dict:
    
    if dict.get("edits") is None:
        return dict

    edits = dict["edits"][0].updates
    ids = edits[~edits["plantingSpaceGlobalId"].isna()]["plantingSpaceGlobalId"]
    if 0 == len(ids):
        return dict

    layerId = 4
    inspections = (
        dict["server"]
        .query(
            [
                LayerQuery(
                    layerId,
                    [
                        "PlantingSpaceGlobalID",
                        "HasActiveWorkOrder",
                    ],
                    "PlantingSpacGlobalID IN ('"
                    + "','".join(str(id) for id in ids)
                    + "')",
                )
            ]
        )[layerId]
        .rename(
            columns={
                "PlantingSpaceGlobalID": "plantingSpaceGlobalId",
            }
        )
    )

    edits = merge(inspections, edits, on="plantingSpaceGlobalId", how="left")
    edits.loc[
        edits["Status"] == Literal["Closed", "Canceled"], "HasActiveWorkOrder"
    ] = 0

    dict["edits"][layerId](
        LayerEdits(
            layerId,
            updates=edits[
                "OBJECTID",
                "HasActiveWorkOrder",
            ].to_json(orient="records"),
        ),
    )

    print(f"Inspections Updated: {len(inspections)}")
    return dict


def update_associated_platingSpace_Address(dict: dict) -> dict:
    if dict.get("edits") is None:
        return dict

    edits = dict["edits"][0].updates
    ids = edits[~edits["plantingSpaceGlobalId"].isna()]["plantingSpaceGlobalId"]
    if 0 == len(ids):
        return dict

    layerId = 2
    plantingSpaces = (
        dict["server"]
        .query(
            [
                LayerQuery(
                    layerId,
                    [
                        "GlobalID",
                        "BuildingNumber",
                        "StreetName",
                        "CrossStreet1",
                        "CrossStreet2",
                    ],
                    "GlobalID IN ('" + ",".join(str(id) for id in ids) + "')",
                )
            ]
        )[layerId]
        .rename(
            columns={
                "GlobalID": "plantingSpaceGlobalId",
            }
        )
    )

    # Transformer.update(
    #     plantingSpaces,
    #     edits,
    #     "plantingSpaceGlobalId",
    #     {
    #         "BuildingNumber": {"Source": "BuildingNumber"},
    #         "StreetName": {"Source": "StreetName"},
    #         "CrossStreet1": {"Source": "CrossStreet1"},
    #         "CrossStreet2": {"Source": "CrossStreet2"},
    #     },
    # )

    dict["edits"][layerId](
        LayerEdits(
            layerId,
            updates=plantingSpaces.to_json(orient="records"),
        ),
    )

    print(f"Planting Spaces Updated: {len(plantingSpaces)}")
    return dict


def contractGetChanges(
    service: eComply,
    repo: Server,
    serverGen: int,
) -> dict:
    layerId = 1

    def strategy():
        fromDateTime = datetime.fromtimestamp(serverGen)
        edits = service.getContracts(fromDateTime)
        print(f"Contracts Recieved: {len(edits)}")
        return edits

    dict = getChanges(layerId, strategy)
    dict["server"] = repo

    return dict


def workOrderLineItemGetChanges(
    service: eComply,
    repo: Server,
    serverGen: int,
) -> dict:
    layerId = 2

    def strategy():
        fromDateTime = datetime.fromtimestamp(serverGen)
        edits = service.getWorkOrderLineItems(fromDateTime)
        print(f"Line Items Recieved: {len(edits)}")
        return edits

    dict = getChanges(layerId, strategy)
    dict["server"] = repo

    return dict

Pipeline


In [ ]:
proxy = "@bcpxy.nycnet:8080"

import os

# Set proxy incase environment not set
os.environ["HTTP_PROXY"] = proxy
os.environ["HTTPS_PROXY"] = proxy
# bypass proxy on parks domains
os.environ["NO_PROXY"] = ".parks.nycnet"

# arcGIS = "https://formsgisportal.parks.nycnet"
# arcGIS = 'https://stg-formsgisportal.parks.nycnet'
arcGIS = "https://dev-formsgisportal.parks.nycnet"

factory = GISFactory(
    url=arcGIS + "/portal/home",
    username="forms.python_user",
    password="formsPython24*",
)
eComplyRepo = factory.CreateFeature(
    arcGIS + "/server/rest/services/eComply/eComplyContract/FeatureServer"
)
dataPushRepo = factory.CreateFeature(
    arcGIS + "/server/rest/services/DataPush/ForMSDataPush/FeatureServer"
)

service = eComply(
    url="https://nycparks-stage.ecomply.us/WebAPI",
    username="ff@ecomply.us",
    password="!test123",
)

from toolz import pipe

# Push DomainValues
# dataPushRepo = factory.CreateFeature(arcGIS + 'server/rest/services/DataPush/ForMSDataPush/FeatureServer')
# domainValues = dataPushRepo.queryDomains([LayerDomainNames(0, ['WOContract','WOProject','WOEntity','WOStatus','WOCategory','WOType','GenusSpecies'])])
# domains = [
#     {
#         'domainName': domain['name'],
#         'code': str(value['code']),
#         'value': value['name'],
#     }
#     for domain in domainValues for value in domain['codedValues']
# ]
# service.postDomainValues(domains)
# print('DomainValues Pushed')

import time

seconds = 60 * 60 * 24
epoch = time.time() - seconds
serverGens = DataFrame({"Contract": [epoch], "WorkOrder": [epoch]})

# serverGens = eComplyRepo.query([LayerQuery(3, ["*"])])[3]
newServerGens = serverGens.copy()
print(newServerGens)

# sendContracts = pipe(
#     contractExtractChanges(eComplyRepo, serverGens.at[0, 'Contract'], service),
#     contractSendEdits,
# )
# newServerGens['Contract'] = sendContracts['serverGen']
# print(newServerGens)
# eComplyRepo.apply_edits([LayerEdits(3, updates=newServerGens.to_json(orient="records"))])

sendWorkOrders = pipe(
    workOrderExtractChanges(dataPushRepo, serverGens.at[0, "WorkOrder"], service),
    hydrateInspection,
    hydratePlantingSpace,
    workOrderSendEdits,
)
# newServerGens['WorkOrder'] = sendWorkOrders['serverGen']
# print(newServerGens)
# eComplyRepo.apply_edits([LayerEdits(3, updates=newServerGens.to_json(orient="records"))])

# applyWorkOrders = pipe(
#     workOrderGetChanges(service, dataPushRepo, serverGens.at[0, 'WorkOrder']),
#     update_associated_inspection_HasActiveWorkOrder,
#     update_associated_platingSpace_Address,
#     applyEdits
# )

# applyContracts = pipe(
#     contractGetChanges(service, eComplyRepo, serverGens.at[0, 'Contract']),
#     applyEdits
# )

# applyWorkOrderLineItems = pipe(
#     workOrderLineItemGetChanges(service, dataPushRepo, serverGens.at[0, 'WorkOrder']),
#     applyEdits
# )